In [1]:
USER_FLAG = "--user"

In [2]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

     |████████████████████████████████| 1.6 MB 7.8 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 296 kB 6.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 135 kB 51.5 MB/s            
     |████████████████████████████████| 127 kB 43.9 MB/s            
     |████████████████████████████████| 636 kB 34.5 MB/s            
     |████████████████████████████████| 1.6 MB 37.9 MB/s            
     |████████████████████████████████| 61 kB 46 kB/s              
     |████████████████████████████████| 152 kB 46.4 MB/s            
     |████████████████████████████████| 54 kB 5.0 MB/s             
     |████████████████████████████████| 52 kB 2.6 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 56 kB 6.8 MB/s             
  Preparing metadata (setup.py) ... done
  Instal

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.9
google_cloud_pipeline_components version: 0.2.0


In [2]:
import os
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  niveustraining


In [3]:
BUCKET_NAME="gs://" + "black-friday-demo" + "-bucket"

In [4]:
import kfp

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from typing import NamedTuple

In [5]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline-root/"
PIPELINE_ROOT

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://black-friday-demo-bucket/pipeline-root/'